In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Cargamos los datos
df = pd.read_csv('output_steam_games_final.csv')

# Elimina las filas con valores NaN en la columna de juegos
df = df.dropna(subset=['title'])

# Creamos una matriz de características utilizando CountVectorizer
count = CountVectorizer()
count_matrix = count.fit_transform(df['title'])

# Calculamos la similitud del coseno
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Creamos una serie para mapear los índices de los juegos a sus títulos
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def recomendacion_juego(titulo, cosine_sim=cosine_sim):
    # Obtenemos el índice del juego que coincide con el título
    idx = indices[titulo]

    # Obtenemos las puntuaciones de similitud por pares de todos los juegos con ese juego
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenamos los juegos en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenemos las puntuaciones de los 5 juegos más similares
    sim_scores = sim_scores[1:6]

    # Obtenemos los índices de los juegos
    juego_indices = [i[0] for i in sim_scores]

    # Devolvemos los 5 juegos más similares
    return df['title'].iloc[juego_indices]


In [3]:
# Supongamos que tienes un juego llamado 'Super Juego' en tu conjunto de datos
juego = 'Robo Miner'

# Usa la función para obtener recomendaciones
recomendaciones = recomendacion_juego(juego)

# Imprime las recomendaciones
print(recomendaciones)


16057        Business-hooiznes
18978         Geology Business
1965       ComiPo! Casual Wear
1966         ComiPo! Swim Wear
5048     Suits: A Business RPG
Name: title, dtype: object


2do modelo de recomendacion

In [18]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Carga los datos
df = pd.read_csv('2dafuncion_final.csv')

# Crea la matriz de utilidad usando 'playtime_forever' como valor
utilidad = df.pivot_table(index='user_id', columns='title', values='playtime_forever')

# Calcula la similitud del coseno
similitud = cosine_similarity(utilidad.fillna(0))

# Crea un mapeo de ID de usuario a índice de matriz
user_id_to_index = {user_id: index for index, user_id in enumerate(utilidad.index)}

# Sistema de recomendación usuario-ítem
def recomendacion_usuario(user_id):
    # Obtiene el índice de la matriz para el ID de usuario
    user_index = user_id_to_index[user_id]
    # Obtiene los índices de los 6 usuarios más similares
    indices_similares = np.argsort(similitud[user_index])[-7:-1][::-1]
    # Encuentra los juegos que les gustaron a los usuarios similares
    juegos_recomendados = utilidad.iloc[indices_similares].mean().sort_values(ascending=False).index[:6]
    # Excluye el primer juego (el más recomendado) y devuelve los siguientes 5
    return juegos_recomendados[1:]

# Asegúrate de que 'user_id' es un ID de usuario válido en tus datos
user_id = 'Joanfortes'  # Reemplaza con un ID de usuario válido de tus datos

if user_id in user_id_to_index:
    # Llama a la función de recomendación de usuario
    juegos_recomendados = recomendacion_usuario(user_id)

    # Imprime los juegos recomendados
    print("Juegos recomendados para el usuario {}: {}".format(user_id, juegos_recomendados))
else:
    print("El ID de usuario {} no se encuentra en los datos.".format(user_id))






Juegos recomendados para el usuario Joanfortes: Index(['Torchlight II', '10,000,000', '100% Orange Juice',
       '12 Labours of Hercules II: The Cretan Bull', '140'],
      dtype='object', name='title')
